# Transcriptome analysis 101

Here is a simple example of transcriptomic analysis

## Install essential analytic and visualisation packages


4 packages are needed for this tutorial
* [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html)
* [ashr](https://cran.r-project.org/web/packages/ashr/), needed by DESeq2
* [EnhancedVolcano](https://bioconductor.org/packages/release/bioc/html/EnhancedVolcano.html)
* [pheatmap](https://cran.r-project.org/web/packages/pheatmap/)

If you need to install those packages. The current version of R and Jupyter already have those installed.

In [ ]:
#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager");
#
# Install DESeq2 (Transcriptome analysis)
#BiocManager::install("DESeq2");
#
# Install EnhancedVolcano (Volcano plot)
#BiocManager::install("EnhancedVolcano");
#
# Install ashr (Empirical Bayes approach for large-scale hypothesis testing and false discovery rate (FDR) estimation)
#install.packages("ashr", quietly = TRUE);
#
# Install pheatmap (Pretty heatmaps)
#install.packages("pheatmap", quietly = TRUE);

## Load package and datasets

In [ ]:
library("DESeq2");
library("ashr");
library("EnhancedVolcano");
library("pheatmap");

In [ ]:
countData <- as.matrix(read.csv("gene_count_matrix.csv", row.names="gene_id"));
colData <- read.csv("pheno.csv", sep=",", row.names=1);

In [ ]:
# First 5 row of transcriptomic data. For each gene (row), the absolute expression level for each sample (KPRM)
head(countData);

In [ ]:
# Experimental conditions (Natural light = Control)
colData;

## Process the dataset

Process the dataset, associate expression level and condition. Explicitly fix control condition as `natural`.

In [ ]:
dds <- DESeqDataSetFromMatrix(countData = countData, colData = colData, design = ~ condition);
dds$condition <- relevel(dds$condition, ref = "natural");
dds <- DESeq(dds);

Run a [PCA analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) of the normalised data

In [ ]:
vsd <- vst(dds);
plotPCA(vsd, intgroup=c("condition"));

Analyse the data. Compare `light` (24/7) vs Control (Natural cycle)

In [ ]:
res <- lfcShrink(dds, contrast=c("condition","light","natural"), type="ashr");
head(res);

How many gene are differentialy expressed?
`padj` is is P-value (`pvalue`) corrected for multiple testing. So we want to use that. `padj < 0.001`.

In [ ]:
sum(res$padj < 0.001, na.rm=TRUE); # ignore the empty results (gene with not enough expression data)

Also we can check the expression change with `log2FoldChange`. This is the log(2) of the change of expression. So a 2x increase of expression is log2(2) = 1 while a 3x decrease woulv be -log2(3) = -1.58496250072116.  

In [ ]:
sum(res$padj < 0.001 & abs(res$log2FoldChange) >= log2(3), na.rm=TRUE); # Change 2x (increase and decrease)

## Visualise

Visualise the results with a _classic_ [volcano plot](https://en.wikipedia.org/wiki/Volcano_plot_(statistics)) using `EnhancedVolcano`. Here is a [tutorial](https://www.bioconductor.org/packages/release/bioc/vignettes/EnhancedVolcano/inst/doc/EnhancedVolcano.html) is you want to change the output.

In [ ]:
EnhancedVolcano(res, lab = rownames(res), pCutoff = 0.001, FCcutoff = 3, x = 'log2FoldChange', y = 'padj', xlim = c(-12, 8),title="light vs natural");

Another classic view is an [heatmap](https://en.wikipedia.org/wiki/Heat_map) view that associated the data by conditions. Here is a [tutorial](https://davetang.org/muse/2018/05/15/making-a-heatmap-in-r-with-the-pheatmap-package/) is you want to change the apparence.

In [ ]:
df <- as.data.frame(colData(vsd)[,c("condition")]);
rownames(df) <- colnames(assay(vsd));
pheatmap(assay(vsd)[which(res$padj < 0.001 & abs(res$log2FoldChange) >= log2(3)),], cluster_rows=TRUE, show_rownames=FALSE, cluster_cols=FALSE, annotation_col=df, scale="row", border_color="white");

## Save and extract genes of interest

Re-order the gene list by adjusted P-value and save it.

In [ ]:
resOrdered <- res[order(res$padj), ];
write.csv(as.data.frame(resOrdered), file="condition_light_vs_natural.csv");

Make a list of the genes of interest

In [ ]:
light_list <- which(res$padj < 0.001 & abs(res$log2FoldChange) >= log2(3));
light_list

Plot the gene with be lowest adjusted P-value.

In [ ]:
plotCounts(dds, gene=which.min(res$padj), intgroup="condition");

## Misc.

In [ ]:
# Normalised
head(assay(vsd)[light_list,]);

In [ ]:
# Not normlised
head(assay(dds)[light_list,]);